In [1]:
!pip install fasttext


  Using cached https://files.pythonhosted.org/packages/10/61/2e01f1397ec533756c1d893c22d9d5ed3fce3a6e4af1976e0d86bb13ea97/fasttext-0.9.1.tar.gz
  Running setup.py bdist_wheel for fasttext ... error
  Complete output from command /srv/paws/bin/python3.6 -u -c "import setuptools, tokenize;__file__='/tmp/pip-build-u7h5buj1/fasttext/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d /tmp/tmp4cevh2nppip-wheel- --python-tag cp36:
  usage: -c [global_opts] cmd1 [cmd1_opts] [cmd2 [cmd2_opts] ...]
     or: -c --help [cmd1 cmd2 ...]
     or: -c --help-commands
     or: -c cmd --help
  
  error: invalid command 'bdist_wheel'
  
  ----------------------------------------
  Failed building wheel for fasttext
  Running setup.py clean for fasttext
Failed to build fasttext
  Running setup.py install for fasttext ... done


In [2]:
import numpy as np
import fasttext
model=fasttext.load_model('model.bin')


def qid_to_claims(qid):
    
    import requests

    S = requests.Session()

    URL = "http://www.wikidata.org/w/api.php"
    PARAMS = {
    "action": "wbgetentities",
    "ids": qid,
    "props": "claims",
    "format": "json"
         }

    R = S.get(url=URL, params=PARAMS)
    DATA = R.json()
    dict=DATA['entities'][qid]['claims'].keys()

    Claims=[]

    for i in dict:
         length= len(DATA['entities'][qid]['claims'][i])
    
         for j in range(0,length):
            if('datavalue' in DATA['entities'][qid]['claims'][i][j]['mainsnak'].keys() ):
                if(DATA['entities'][qid]['claims'][i][j]['mainsnak']['datavalue']['type']=='wikibase-entityid'):
                    Claims.append(tuple((i, DATA['entities'][qid]['claims'][i][j]['mainsnak']['datavalue']['value']['id'])))
            
                else:
                    Claims.append(tuple((i,"")))

    return Claims

def evaluate_with_multiple_pid(qid1,qid2,pid_set): #general function of cosine similarity which takes two Qids.
    prop1=qid_to_claims(qid1)
    prop2=qid_to_claims(qid2)

    embedding1=[]
    embedding2=[]
    count1=0
    count2=0
    
    for i in range(len(prop1)):
        if(prop1[i][0] in pid_set):
                
            
                
            pid=prop1[i][0]
            qid=prop1[i][1]
        
            if count1==0:
                embedding1=np.array(model.get_word_vector(pid))
            else:
                embedding1=np.add(embedding1,np.array(model.get_word_vector(pid)))
            count1=count1+1
            if qid!='':
                embedding1=np.add(embedding1,np.array(model.get_word_vector(qid)))
                count1=count1+1
    doc_embedding1=np.divide(embedding1,count1) #document embedding of 1st property

    for i in range(len(prop2)):
        if(prop2[i][0] in pid_set):
                
            pid=prop2[i][0]
            qid=prop2[i][1]
        
            if count2==0:
                embedding2=np.array(model.get_word_vector(pid))
            else:
                embedding2=np.add(embedding2,np.array(model.get_word_vector(pid)))
            count2=count2+1
            if qid!='':
                embedding2=np.add(embedding2,np.array(model.get_word_vector(qid)))
                count2=count2+1
    doc_embedding2=np.divide(embedding2,count2) #document embedding of 2nd property

    sq1=0
    sq2=0
    sum=0
    for i in range(50):
        sum=sum+(doc_embedding1[i]*doc_embedding2[i])
        sq1=sq1+(doc_embedding1[i]**2)
        sq2=sq2+(doc_embedding2[i]**2)
    sq1=sq1**0.5
    sq2=sq2**0.5

    cosine_similarity=sum/(sq1*sq2) #comparing 2 document embeddings using cosine similarity

    return cosine_similarity


In [3]:
#comparing China and india with properties- P37(official language) and P30(continent)

china='Q148'
india='Q668'
multiple_pids=set(["P37","P30"])

print(evaluate_with_multiple_pid(china,india,multiple_pids))

0.6638342133450517


In [17]:
#comparing japan and nepal with properties- P37(official language) and P30(continent)

japan='Q17'
nepal='Q837'
multiple_pids=set(["P37","P30"])

print(evaluate_with_multiple_pid(china,india,multiple_pids))

0.6638342133450517


In [14]:
#comparing China and india with properties- P47(shared border with.. ), P37(official language), and P30(continent)

japan='Q17'
india='Q668'
multiple_pids=set(["P47","P37","P30"])

print(evaluate_with_multiple_pid(china,india,multiple_pids))

0.8748522287250468


In [4]:
#comparing greenland and india with properties- P37(official language) and P30(continent)

greenland='Q223'
india='Q668'
multiple_pids=set(["P37","P30"])

print(evaluate_with_multiple_pid(greenland,india,multiple_pids))


-0.022140771930297856


In [18]:
#comparing greenland and india with properties- P47(shared border with.. ), P37(official language) and P30(continent)

greenland='Q223'
india='Q668'
multiple_pids=set(["P37","P30","P47"])

print(evaluate_with_multiple_pid(greenland,india,multiple_pids))

0.015427982408300301


In [25]:
#comparing Cristiano_Ronaldo and Neymar with properties- award received (P166), member of sports team (P54) and
# number of matches played/races/starts (P1350)

Cristiano_Ronaldo="Q11571"
Neymar="Q142794"
multiple_pids=set(["P54","P166","P1350"])

print(evaluate_with_multiple_pid(Cristiano_Ronaldo,Neymar,multiple_pids))

0.6796209228715867
